In [11]:
library(ape)
library(ggtree)
library(ggplot2)
library(gridExtra)
library(dplyr)
library(gtable)
library(grid)

# ---------------------------
# Read and prepare data
# ---------------------------
read_trait <- function(path) {
  con <- file(path, "r")
  header <- readLines(con, n = 1)
  close(con)
  df <- read.table(path, header = FALSE, skip = 1)
  colnames(df)[1] <- "species"
  rownames(df) <- df$species
  df$species <- NULL
  return(df)
}

species_map <- c(
  "Frog"    = "Xenopus_tropicalis",
  "Pig"     = "Sus_scrofa",
  "Rat"     = "Rattus_norvegicus", 
  "Mouse"   = "Mus_musculus",
  "Human"   = "Homo_sapiens",
  "Macaque" = "Macaca_mulatta"
)

# Dictionary for proper species labeling
species_display_names <- c(
  "Xenopus_tropicalis" = "Xenopus tropicalis",
  "Sus_scrofa" = "Sus scrofa", 
  "Rattus_norvegicus" = "Rattus norvegicus",
  "Mus_musculus" = "Mus musculus",
  "Homo_sapiens" = "Homo sapiens",
  "Macaca_mulatta" = "Macaca mulatta"
)

# Read trait data
trait_b <- read_trait("trait_b")
trait_gamma <- read_trait("trait_gamma")
trait_beta <- read_trait("trait_beta")

# Apply species mapping
rownames(trait_b) <- species_map[rownames(trait_b)]
rownames(trait_gamma) <- species_map[rownames(trait_gamma)]
rownames(trait_beta) <- species_map[rownames(trait_beta)]

# Function to prepare trait data
prepare_trait_data <- function(trait_matrix, trait_name) {
  trait_long <- data.frame()
  for(sp in rownames(trait_matrix)) {
    sp_values <- as.numeric(trait_matrix[sp, ])
    sp_values_clean <- sp_values[!is.na(sp_values)]
    
    if(length(sp_values_clean) > 0) {
      sp_data <- data.frame(
        species = sp,
        value = sp_values_clean,
        trait = trait_name
      )
      trait_long <- rbind(trait_long, sp_data)
    }
  }
  return(trait_long)
}

# Prepare all trait data
trait_b_long <- prepare_trait_data(trait_b, "b")
trait_gamma_long <- prepare_trait_data(trait_gamma, "gamma")
trait_beta_long <- prepare_trait_data(trait_beta, "beta")

# Combine and add Greek symbols
all_traits_long <- rbind(trait_b_long, trait_gamma_long, trait_beta_long)
all_traits_long$trait_greek <- factor(all_traits_long$trait,
                                      levels = c("b", "gamma", "beta"),
                                      labels = c("b", "γ", "β"))

# ---------------------------
# Create publication-quality tree plot with proper species names
# ---------------------------
newick_tree <- "(Xenopus_tropicalis:351.68654000,(Sus_scrofa:94.00000000,((Rattus_norvegicus:11.64917000,Mus_musculus:11.64917000)'14':75.55083000,(Homo_sapiens:28.82000000,Macaca_mulatta:28.82000000)'13':58.38000000)'25':6.80000000)'37':257.68654000);"
tree <- read.tree(text = newick_tree)

# Create a clean, publication-quality tree with proper species names
tree_plot <- ggtree(tree, size = 1) + 
  geom_tiplab(aes(label = species_display_names[label]), 
              size = 5, hjust = 0, fontface = "italic", offset = 5) +
  geom_treescale(width = 100, x = 50, y = 1.7, fontsize = 4, linesize = 1, offset = .2) +  # Moved scale further right
  xlim(0, 500) +
  theme(plot.margin = margin(10, 10, 10, 15))

# ---------------------------
# Create publication-quality boxplot
# ---------------------------
# Order species by tree tip order (reverse for proper vertical alignment)
tree_tip_order <- rev(tree$tip.label)
all_traits_long$species <- factor(all_traits_long$species, levels = tree_tip_order)

# Publication-quality boxplot
combined_boxplot <- ggplot(all_traits_long, aes(x = species, y = value, fill = trait_greek)) +
  geom_boxplot(alpha = 0.9, width = 0.7, outlier.size = 1, outlier.alpha = 0.6) +
  coord_flip() +
  labs(x = "", y = "Trait Value", fill = NULL) +
  scale_fill_manual(
    name = NULL,
    values = c("b" = "#4E79A7", "γ" = "#F28E2B", "β" = "#59A14F"),
    labels = expression(log[10](b), log[10](beta), log[10](gamma))
  ) +
  theme_minimal(base_size = 12) +
  theme(
    axis.text.y = element_blank(),
    axis.title.x = element_text(size = 12, face = "bold", margin = margin(t = 10)),
    axis.text.x = element_text(size = 10),
    legend.position = "bottom",
    legend.text = element_text(size = 14, face = "bold"),
    legend.margin = margin(t = 5, b = 5),
    legend.title = element_blank(),
    plot.margin = margin(10, 20, 10, 10),
    panel.grid.major.y = element_blank(),
    panel.grid.minor.y = element_blank(),
    panel.grid.major.x = element_line(color = "grey90", linewidth = 0.5),
    panel.grid.minor.x = element_blank(),
    plot.background = element_rect(fill = "white", color = NA)
  ) +
  scale_x_discrete(drop = FALSE)

# ---------------------------
# Perfect alignment using gtable
# ---------------------------
# Convert to grobs
tree_grob <- ggplotGrob(tree_plot)
boxplot_grob <- ggplotGrob(combined_boxplot)

# Ensure both plots have the same height
max_height <- unit.pmax(tree_grob$heights[2:length(tree_grob$heights)], 
                        boxplot_grob$heights[2:length(boxplot_grob$heights)])

tree_grob$heights[2:length(tree_grob$heights)] <- max_height
boxplot_grob$heights[2:length(boxplot_grob$heights)] <- max_height

# Combine everything
final_plot <- arrangeGrob(
  tree_grob,
  boxplot_grob,
  ncol = 2,
  widths = c(2.5, 1.5)
)

# ---------------------------
# Save figure
# ---------------------------
# High-resolution PDF
pdf("Figure2.pdf", width = 12, height = 8, useDingbats = FALSE)
grid.draw(final_plot)
dev.off()

# cat("\nSession info for reproducibility:\n")
# sessionInfo()

Warning message in fortify(data, ...):
“Arguments in `...` must be used.
✖ Problematic arguments:
• as.Date = as.Date
• yscale_mapping = yscale_mapping
• hang = hang
• size = 1
ℹ Did you misspell an argument name?”


agg_record_1023670946 
                    2